# Models

> The models for our backend

In [1]:
#| default_exp db.models

In [2]:
#| export
from uuid import UUID, uuid4
from enum import StrEnum
import typing as t
from datetime import datetime

from sqlmodel import Field, SQLModel, Column, JSON

In [3]:
#| export
class TaskStates(StrEnum):
    PENDING = "pending"
    CLAIMED = "claimed"
    COMPLETED = "completed"

In [4]:
TaskStates.CLAIMED

<TaskStates.CLAIMED: 'claimed'>

In [5]:
#| export

class Task(SQLModel, table=True):
    id: UUID = Field(default_factory=uuid4, primary_key=True)
    state: TaskStates = Field(default=TaskStates.PENDING)
    name: t.Optional[str] = None
    added: datetime = Field(default_factory=datetime.utcnow)
    finished: t.Optional[datetime] = Field(default=None)
    payload: t.Dict[str, t.Any] = Field(default={}, sa_column=Column(JSON))

In [6]:
Task()

Task(id=UUID('c671bb0c-5447-484e-b6e4-26f218ad81b4'), state=<TaskStates.PENDING: 'pending'>, name=None, added=datetime.datetime(2025, 3, 16, 19, 57, 14, 506342), finished=None, payload={})

In [7]:
from openSQS.db.connection import get_test_session, init_db

init_db()

with get_test_session() as session:
    new_task = Task(name="test")
    session.add(new_task)
    session.commit()

2025-03-16 12:57:14,606 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-03-16 12:57:14,606 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-16 12:57:14,607 INFO sqlalchemy.engine.Engine select current_schema()
2025-03-16 12:57:14,608 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-16 12:57:14,608 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-03-16 12:57:14,609 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-16 12:57:14,609 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-16 12:57:14,610 INFO sqlalchemy.engine.Engine INSERT INTO task (id, state, name, added, finished, payload) VALUES (%(id)s::UUID, %(state)s, %(name)s, %(added)s, %(finished)s, %(payload)s::JSON)
2025-03-16 12:57:14,610 INFO sqlalchemy.engine.Engine [generated in 0.00018s] {'id': UUID('0ed039c7-e198-40d4-9ab5-9081fec57b47'), 'state': 'PENDING', 'name': 'test', 'added': datetime.datetime(2025, 3, 16, 19, 57, 14, 580311), 'finished': None, 'payload': '{}'}
2025-03-16 12:57

ProgrammingError: (psycopg2.errors.UndefinedTable) relation "task" does not exist
LINE 1: INSERT INTO task (id, state, name, added, finished, payload)...
                    ^

[SQL: INSERT INTO task (id, state, name, added, finished, payload) VALUES (%(id)s::UUID, %(state)s, %(name)s, %(added)s, %(finished)s, %(payload)s::JSON)]
[parameters: {'id': UUID('0ed039c7-e198-40d4-9ab5-9081fec57b47'), 'state': 'PENDING', 'name': 'test', 'added': datetime.datetime(2025, 3, 16, 19, 57, 14, 580311), 'finished': None, 'payload': '{}'}]
(Background on this error at: https://sqlalche.me/e/20/f405)